In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np



In [2]:
device="cuda" if torch.cuda.is_available() else "cpu"
model_name="gpt2-xl"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Although Transformers provides a generate() func‐
tion for autoregressive models like GPT-2, we’ll implement this decoding method ourselves to see what goes on under the hood.

In [3]:
prompt = "Transformers are the"

input_ids=tokenizer(prompt, return_tensors='pt' )['input_ids'].to(device)
output=model(input_ids=input_ids)
logits=output.logits
next_token_logitId=torch.argmax(logits[0][-1])
tokenizer.decode(next_token_logitId)

' most'

In [4]:
prompt = "Transformers are the"

input_ids=tokenizer(prompt, return_tensors='pt' )['input_ids'].to(device)
output=model(input_ids=input_ids)
logits=output.logits
probas=torch.softmax(logits[0,-1,:],dim=-1)
next_token_probId = torch.argmax(probas)
tokenizer.decode(next_token_probId)

' most'

In [5]:
next_token_logit=logits[0,-1,:]
top_k=torch.topk(next_token_logit,k=5)
tokenizer.decode(top_k.indices)

' most only best Transformers ultimate'

In [6]:
# using model.generate() method to generate new text
input_txt = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
model.to(device)
output = model.generate(input_ids,max_new_tokens=120, 
                        pad_token_id=tokenizer.eos_token_id,  # Explicitly set pad_token_id
                        attention_mask=torch.ones_like(input_ids),
                        do_sample=False
                       )
print(tokenizer.decode(output[0]))

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.


The researchers, from the University of California, Davis, and the University of Colorado, Boulder, were conducting a study on the Andean cloud forest, which is home to the rare species of cloud forest trees.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.


The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The researchers were surprised to find that the unicorns were able to communicate with each other, and even with humans.

The


#### Drawbacks with greedy search decoding:
It tends to produce repeti‐
tive output sequences, which is certainly undesirable in a news article. This is a com‐
mon problem with greedy search algorithms, which can fail to give you the optimal
solution; in the context of decoding, they can miss word sequences whose overall
probability is higher just because high-probability words happen to be preceded by
low-probability ones.

In [7]:
input_ids[0]

tensor([  818,   257, 14702,  4917,    11, 11444,  5071,   257, 27638,   286,
        28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
        19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
         6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
        19942,  5158,  2818,  3594,    13,   628,   198], device='cuda:0')